<a href="https://colab.research.google.com/github/Aasha03/fmllab/blob/main/Classifi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Define the path to the parent folder containing your dataset subfolders
parent_folder_path = '/content/drive/MyDrive/water'

# Define transformations including resizing and data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images up to 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
])

# Load the dataset
dataset = ImageFolder(root=parent_folder_path, transform=transform)

In [9]:
from torch.utils.data import random_split

# Define the sizes for train, validation, and test sets
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [10]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify the output layer to match the number of classes (5 in your case)
num_classes = 6
model.fc = nn.Linear(model.fc.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
import torch.optim as optim
from torch.utils.data import DataLoader

# Define hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training loss for the current epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

Epoch 1/10, Loss: 1.2088526537021
Epoch 2/10, Loss: 0.8678620010614395
Epoch 3/10, Loss: 0.6064199308554331
Epoch 4/10, Loss: 0.55506515254577
Epoch 5/10, Loss: 0.5635991841554642
Epoch 6/10, Loss: 0.374998373289903
Epoch 7/10, Loss: 0.3426751097043355
Epoch 8/10, Loss: 0.2070964133987824
Epoch 9/10, Loss: 0.13668343052268028
Epoch 10/10, Loss: 0.17917765521754822


In [15]:
import torch
import random
import numpy as np
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define data transforms and augmentations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load your dataset
dataset = ImageFolder('/content/drive/MyDrive/water', transform=transform)

# Split dataset into training, validation, and test sets
train_size = 0.8
val_size = 0.1
test_size = 0.1

train_indices, temp_indices = train_test_split(list(range(len(dataset))), test_size=(1 - train_size), random_state=seed)
val_indices, test_indices = train_test_split(temp_indices, test_size=test_size / (val_size + test_size), random_state=seed)

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=64, sampler=test_sampler)

# Define your model architecture (ResNet-18 in this example)
model = resnet18(pretrained=True)
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the selected device (GPU or CPU)
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

# Training loop with early stopping
best_val_accuracy = 0.0
patience = 5
early_stopping_counter = 0

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_accuracy = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            val_accuracy += (predicted == labels).sum().item()

    val_accuracy /= len(val_loader.dataset)

    # Adjust learning rate based on validation accuracy
    scheduler.step(val_accuracy)

    print(f'Epoch [{epoch + 1}/{num_epochs}] Val Accuracy: {val_accuracy:.4f}')

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print("Early stopping")
        break

# Evaluation on the test set
model.eval()
test_accuracy = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_accuracy += (predicted == labels).sum().item()

test_accuracy /= len(test_loader.dataset)

print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch [1/20] Val Accuracy: 0.0417
Epoch [2/20] Val Accuracy: 0.0322
Epoch [3/20] Val Accuracy: 0.0322
Epoch [4/20] Val Accuracy: 0.0568
Epoch [5/20] Val Accuracy: 0.0625
Epoch [6/20] Val Accuracy: 0.0720
Epoch [7/20] Val Accuracy: 0.0530
Epoch [8/20] Val Accuracy: 0.0473
Epoch [9/20] Val Accuracy: 0.0606
Epoch 00010: reducing learning rate of group 0 to 1.0000e-04.
Epoch [10/20] Val Accuracy: 0.0625
Epoch [11/20] Val Accuracy: 0.0758
Epoch [12/20] Val Accuracy: 0.0795
Epoch [13/20] Val Accuracy: 0.0777
Epoch [14/20] Val Accuracy: 0.0777
Epoch [15/20] Val Accuracy: 0.0795
Epoch [16/20] Val Accuracy: 0.0852
Epoch [17/20] Val Accuracy: 0.0852
Epoch [18/20] Val Accuracy: 0.0720
Epoch [19/20] Val Accuracy: 0.0777
Epoch 00020: reducing learning rate of group 0 to 1.0000e-05.
Epoch [20/20] Val Accuracy: 0.0777
Test Accuracy: 0.0777


In [24]:
pip install onnx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 95.0 MB/s eta 0:00:00


In [25]:
import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)


In [32]:
import torch
from torchvision import models

# Load your ResNet model (modify the architecture and weights loading as needed)
model = models.resnet50()
model.load_state_dict(torch.load('/content/drive/MyDrive/model/my_model_weights.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [33]:
import torch.onnx as torch_onnx

# Provide example input dimensions (modify according to your model's input)
dummy_input = torch.randn(1, 3, 224, 224)

# Specify the path where you want to save the ONNX model
onnx_path = '/content/drive/MyDrive/onnx/my_model.onnx'

# Export the model to ONNX format
torch_onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=['input'], output_names=['output'])

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import torch
from torchvision import models
import torch.onnx as torch_onnx
from google.colab import files

# Load your ResNet model (modify the architecture and weights loading as needed)
model = models.resnet50()
model.load_state_dict(torch.load('/content/drive/MyDrive/model/my_model_weights.pth'))
model.eval()

# Provide example input dimensions (modify according to your model's input)
dummy_input = torch.randn(1, 3, 224, 224)

# Specify the path where you want to save the ONNX model
onnx_path = 'your_model.onnx'

# Export the model to ONNX format
torch_onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=['input'], output_names=['output'])

# Download the ONNX model to your local machine
files.download(onnx_path)